In [67]:
import json
from pathlib import Path
import os
import numpy as np

In [2]:
path = "/home/weedsci/matt/SemiF-AnnotationPipeline/data/semifield-utils/species_info.json"
with open(path, 'r') as f:
  data = json.load(f)
collection_location = ""
for key in data["species"].keys():
    old_collection_location = collection_location
    collection_location = data["species"][key]["collection_location"]
    if old_collection_location != collection_location:
        print(collection_location)
    print(key, data["species"][key]["common_name"])

NC
AMPA Palmer amaranth
AMAR2 Common ragweed
SEOB4 Sicklepod
XAST Cocklebur
DISA Large crabgrass
ELIN3 Goosegrass
URPL2 Broadleaf signalgrass
CYRO Purple nutsedge
AMTU Waterhemp
ECCR Barnyardgrass
ECCO2 Jungle rice
URTE2 Texas millet
BASC5 Kochia
HEAN3 Common sunflower
PAHY Ragweed parthenium
SOHA Johnsongrass
GLMA4 Soybean
MD
AMHY Smooth pigweed
CHAL7 Common lambsquarters
PADI Fall panicum
DAST Jimson weed
ABTH Velvetleaf
SEPU8 Yellow foxtail
SEFA Giant foxtail
ERCA20 Horseweed
ZEA Maize
unknown
plant unknown


In [5]:
import json
import os
import sys

from tqdm import tqdm

sys.path.append("/home/weedsci/matt/SemiF-AnnotationPipeline")
from dataclasses import asdict
from pathlib import Path

import pandas as pd
from semif_utils.datasets import Cutout

In [6]:
path = "/home/weedsci/matt/SemiF-AnnotationPipeline/data/semifield-cutouts/MD_2022-07-05/MD_Row-10_1657032952_6.json"
cutoutdir = "/home/weedsci/matt/SemiF-AnnotationPipeline/data/semifield-cutouts"
batch_id = "MD_2022-07-05"
csv_path = "MD_2022-07-05.csv"

In [7]:
def get_cutout_meta(path):
    with open(path) as f:
        j = json.load(f)
        cutout = Cutout(**j)
    return cutout

In [80]:
def cutoutmeta2csv(cutoutdir, batch_id, csv_savepath, save_df=True):
    # Get all json files
    metas = [x for x in Path(cutoutdir, batch_id).glob("*.json")]
    cutouts = []
    for meta in tqdm(metas):
        # Get dictionaries
        cutout = asdict(get_cutout_meta(meta))
        
        row = cutout["cutout_props"]
        
        cls = cutout["cls"]
        colors = cutout["cls"]["rgb"]
        # Extend nested dicts to single column header
        for ro in row:
            rec = {ro: row[ro]}
            cutout.update(rec)
            for cl in cls:
                spec = {cl: cls[cl]}
                if cl == "rgb":
                    r = {"r": str(spec["rgb"][0])}
                    g = {"g": str(spec["rgb"][1])}
                    b = {"b": str(spec["rgb"][2])}
                    cutout.update(r)
                    cutout.update(g)
                    cutout.update(b)

                cutout.update(spec)
                # for idx, color in enumerate(colors):
                #     colo = {color: colors[idx]}
                #     cutout.update(colo)

        # Remove duplicate nested dicts
        cutout.pop("cutout_props")
        cutout.pop("cls")
        cutout.pop("rgb")
        cutout.pop("local_contours")
        # Create and append df
        cutdf = pd.DataFrame(cutout, index=[0])
        cutouts.append(cutdf)
    # Concat and reset index of main df
    cutouts_df = pd.concat(cutouts)
    cutouts_df = cutouts_df.reset_index()
    cutouts_df.drop(columns="index", inplace=True)
    # Save dataframe
    if save_df:
        cutouts_df.sort_values(by = ['image_id', 'cutout_num'], axis=0, ascending=[True, True], inplace=True,
               kind='quicksort', na_position='first', ignore_index=True, key=None)
        cutouts_df.to_csv(csv_savepath, index=False)
    
    return cutouts_df


cutoutmeta2csv(cutoutdir, batch_id, csv_path)


100%|██████████| 1444/1444 [00:01<00:00, 921.62it/s]


,blob_home,data_root,batch_id,image_id,cutout_num,datetime,cutout_id,cutout_path,is_primary,extends_border,...,axis_major_length,axis_minor_length,centroid0,centroid1,eccentricity,extent,solidity,perimeter,is_green,green_sum
0,data,semifield-cutouts,MD_2022-07-05,MDX03535,0,2022:07:06 00:43:43,MDX03535_0,MD_2022-07-05/MDX03535_0.png,False,True,...,640.865423,329.213916,318.955750,254.131675,0.857969,0.240337,0.441101,2855.828786,True,58206
1,data,semifield-cutouts,MD_2022-07-05,MDX03535,1,2022:07:06 00:43:43,MDX03535_1,MD_2022-07-05/MDX03535_1.png,False,False,...,1711.293774,1148.037323,1795.376239,2114.491099,2.436581,1.633181,2.407116,11346.199487,True,728487
2,data,semifield-cutouts,MD_2022-07-05,MDX03535,2,2022:07:06 00:43:43,MDX03535_2,MD_2022-07-05/MDX03535_2.png,False,False,...,928.694196,670.485200,999.517254,971.705462,1.624733,1.108543,1.459556,6125.992529,True,268622
3,data,semifield-cutouts,MD_2022-07-05,MDX03535,3,2022:07:06 00:43:43,MDX03535_3,MD_2022-07-05/MDX03535_3.png,False,True,...,195.950549,30.112565,23.287166,89.663914,0.988122,0.318559,0.660651,446.569589,True,2477
4,data,semifield-cutouts,MD_2022-07-05,MDX03535,4,2022:07:06 00:43:43,MDX03535_4,MD_2022-07-05/MDX03535_4.png,False,True,...,319.740535,58.475860,346.087582,68.707033,1.965322,1.160938,1.773201,750.717821,True,4501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,data,semifield-cutouts,MD_2022-07-05,MD_Row-17_1657033876,10,2022:07:05 23:08:18,MD_Row-17_1657033876_10,MD_2022-07-05/MD_Row-17_1657033876_10.png,False,True,...,1415.404783,774.128834,355.504769,2013.953402,2.419290,1.944804,2.657032,4260.346643,True,367770
1440,data,semifield-cutouts,MD_2022-07-05,MD_Row-17_1657038228,0,2022:07:06 00:20:49,MD_Row-17_1657038228_0,MD_2022-07-05/MD_Row-17_1657038228_0.png,False,True,...,1600.501532,756.102091,476.788295,838.612089,0.881376,0.318115,0.506842,8486.959446,True,496022
1441,data,semifield-cutouts,MD_2022-07-05,MD_Row-17_1657038228,1,2022:07:06 00:20:49,MD_Row-17_1657038228_1,MD_2022-07-05/MD_Row-17_1657038228_1.png,False,True,...,2562.040699,1993.292649,898.342559,1186.274128,0.628252,0.416259,0.608816,19096.700997,True,2191160
1442,data,semifield-cutouts,MD_2022-07-05,MD_Row-17_1657038228,2,2022:07:06 00:20:49,MD_Row-17_1657038228_2,MD_2022-07-05/MD_Row-17_1657038228_2.png,False,True,...,3732.671733,1871.072723,3041.889984,3358.585015,2.582950,1.385122,2.292496,17787.630796,True,1730472


In [75]:
df


,blob_home,data_root,batch_id,image_id,cutout_num,datetime,cutout_id,cutout_path,is_primary,extends_border,...,axis_major_length,axis_minor_length,centroid0,centroid1,eccentricity,extent,solidity,perimeter,is_green,green_sum
0,data,semifield-cutouts,MD_2022-07-05,MD_Row-11_1657028399,39,2022:07:05 21:37:01,MD_Row-11_1657028399_39,MD_2022-07-05/MD_Row-11_1657028399_39.png,False,True,...,92.976435,26.486344,23.498010,47.720864,0.958566,0.542901,0.857212,235.480231,True,1181
1,data,semifield-cutouts,MD_2022-07-05,MD_Row-10_1657028341,4,2022:07:05 21:36:03,MD_Row-10_1657028341_4,MD_2022-07-05/MD_Row-10_1657028341_4.png,False,True,...,1676.915546,619.140887,3197.120699,1302.770351,2.675987,1.794172,2.188482,12298.275606,True,255239
2,data,semifield-cutouts,MD_2022-07-05,MD_Row-11_1657028423,40,2022:07:05 21:37:25,MD_Row-11_1657028423_40,MD_2022-07-05/MD_Row-11_1657028423_40.png,False,True,...,309.286670,132.402654,78.860754,222.621455,0.903736,0.276393,0.484895,1176.472222,True,5262
3,data,semifield-cutouts,MD_2022-07-05,MD_Row-17_1657029191,4,2022:07:05 21:50:13,MD_Row-17_1657029191_4,MD_2022-07-05/MD_Row-17_1657029191_4.png,False,True,...,1890.864324,603.103612,2683.253901,3310.314915,4.343394,2.780504,3.894324,9961.952304,True,254214
4,data,semifield-cutouts,MD_2022-07-05,MD_Row-11_1657037358,2,2022:07:06 00:06:20,MD_Row-11_1657037358_2,MD_2022-07-05/MD_Row-11_1657037358_2.png,False,False,...,1632.116349,820.333874,854.232105,648.103812,0.864508,0.313229,0.567537,8361.721856,True,464320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,data,semifield-cutouts,MD_2022-07-05,MD_Row-16_1657029108,4,2022:07:05 21:48:50,MD_Row-16_1657029108_4,MD_2022-07-05/MD_Row-16_1657029108_4.png,False,True,...,2198.938609,558.629904,608.573942,2775.857558,1.948215,0.314845,0.800604,10913.466156,True,251031
1440,data,semifield-cutouts,MD_2022-07-05,MD_Row-15_1657033612,12,2022:07:05 23:03:53,MD_Row-15_1657033612_12,MD_2022-07-05/MD_Row-15_1657033612_12.png,False,True,...,3170.980184,2373.128391,4135.186531,2461.838467,2.249234,2.239170,2.706039,19395.910017,True,3239808
1441,data,semifield-cutouts,MD_2022-07-05,MD_Row-12_1657033216,4,2022:07:05 22:57:18,MD_Row-12_1657033216_4,MD_2022-07-05/MD_Row-12_1657033216_4.png,False,False,...,641.317224,84.142318,329.036131,156.250453,0.991356,0.171202,0.635674,1800.829653,True,19674
1442,data,semifield-cutouts,MD_2022-07-05,MD_Row-12_1657028531,21,2022:07:05 21:39:13,MD_Row-12_1657028531_21,MD_2022-07-05/MD_Row-12_1657028531_21.png,False,True,...,300.988811,65.922712,56.080080,156.860909,0.975720,0.389749,0.586358,1013.412301,True,6364
